In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer

from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   

def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text) 
    text = text.replace("nbsp", "")
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)

In [3]:
df = pd.read_json('ai_papers.json')

In [4]:
# Preprocessing the dataset
df['abstract'] = df['abstract'].apply(clean_txt)

In [5]:
# Looking at the cleaned dataset
df.head()

,id,title,category,abstract,version
0,0704.0047,Intelligent location of simultaneously active ...,cs.NE cs.AI,intelligent acoustic emission locator describe...,"Sun, 1 Apr 2007 13:06:50 GMT"
1,0704.0050,Intelligent location of simultaneously active ...,cs.NE cs.AI,part describe intelligent acoustic emission lo...,"Sun, 1 Apr 2007 18:53:13 GMT"
2,0704.0304,The World as Evolving Information,cs.IT cs.AI math.IT q-bio.PE,paper discuss benefit describe world informati...,"Wed, 13 Oct 2010 19:49:16 GMT"
3,0704.0671,Learning from compressed observations,cs.IT cs.LG math.IT,problem statistical learn construct predictor ...,"Thu, 5 Apr 2007 02:57:15 GMT"
4,0704.0954,Sensor Networks with Random Links: Topology De...,cs.IT cs.LG math.IT,sensor network practice communication among se...,"Fri, 6 Apr 2007 21:58:52 GMT"


In [6]:
import spacy

In [7]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
%%time
list_docs = []
for i in range(len(df)):
    doc = nlp("u'" + df['abstract'][i] + "'")
    list_docs.append((doc,i))
print(len(list_docs))

In [ ]:
def calculateSimWithSpaCy(nlp, df, user_text, n=6):
    # Calculate similarity using spaCy
    list_sim =[]
    doc1 = nlp("u'" + user_text + "'")
    for i in df.index:
        try:
            doc2 = list_docs[i][0]
            score = doc1.similarity(doc2)
            list_sim.append((doc1, doc2, list_docs[i][1],score))
        except:
            continue

    return  list_sim

In [ ]:
user_query = "face recognition deep learning convolutional neural networks"
user_query = clean_txt(user_query)

In [ ]:
%%time
df3 = calculateSimWithSpaCy(nlp, df, user_query, n=15)

In [ ]:
df_recom_spacy = pd.DataFrame(df3).sort_values([3], ascending=False).head(10)

In [ ]:
df_recom_spacy.reset_index(inplace=True)

In [ ]:
index_spacy = df_recom_spacy[2]
list_scores = df_recom_spacy[3]

In [ ]:
def get_recommendation(top, df_all, scores):
    recommendation = pd.DataFrame(columns = ['Title', 'score'])
    count = 0
    for i in top:
        recommendation.at[count, 'Title'] = df['title'][i]
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    return recommendation

In [ ]:
get_recommendation(index_spacy, df, list_scores)